# The No U-Turn Sampler

Hoffman & Gelman 2011: The No U-Turn Sampler
* implement algorithms 1 (basic HMC) & 6 (NUTS with dual averaging)
* test that your implementations work by replicating the following tests in Section 4.1:
 * 250-dimensional Gaussian
 * Bayesian logistic regression & hierarchical Bayesian logistic regression on the UCI german credit data
 * compare the results given by basic HMC & NUTS
 * _compare your results with the same models run in Stan using the provided NUTS sampler_

# Compare your results with the same models run in Stan using the provided NUTS sampler
## Download  German credit dataset
There are 24 predictors (x). Predictors are normalized to have a zero mean and an unit variance.

Outcome is whether a person should be refused to get a credit or approced to get one. Values for the outcome are -1 and 1, after recoding 0 and 1.

In [26]:
#%reset
import autograd.numpy as np
import autograd.numpy.random as npr
# Download data and standardize X
# Should have values 1 or -1
import pandas as pd
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import pystan
%matplotlib inline
import HMC
import noUturnSampler
import ChainMix
import Neff
import LikelihoodFunctions
#import CovergenceTests
import importlib
importlib.reload(HMC)
importlib.reload(ChainMix)
importlib.reload(noUturnSampler)
importlib.reload(Neff)
importlib.reload(LikelihoodFunctions)

npr.seed(42)
np.seterr('warn')

credit0= pd.read_csv('dataGerman.tab', delim_whitespace=True)

creditY=np.array(credit0['CREDITRATING'])
creditY[creditY==-1]=0

creditX0=np.array(credit0.loc[:, credit0.columns != 'CREDITRATING'])
creditX=(creditX0 - np.mean(creditX0, axis=0))/np.std(creditX0, axis=0)

# Solve beta values for comparison
logistic = LogisticRegression(fit_intercept=False)
betaCoefficients=logistic.fit(creditX,creditY).coef_


## Bayesian logistic regression using German credit data
Target distribution is the posterior of a Bayesian logistic regression model fit to the German credit dataset.

There are 24-beta coefficients associated with the 24 predictors, and intercept alpha. Coefficients and the intercept are given normal priors with variance of 100.

In [27]:
# Bayesian logistic regression
blrModel = """
    data {
      int<lower=0> N;
      int<lower=0> K;   // number of predictors      
      matrix[N,K] x;
      int<lower=0,upper=1> y[N];
    }
    parameters {
      real alpha;
      vector[K] beta;
    }
    model {
      alpha~normal(0,100);
      beta~normal(0,100);  
      y ~ bernoulli_logit(alpha + x*beta);
    }
"""

credit_dat = {'x': creditX,
                'y': creditY,
                'N': len(creditY),
                'K': creditX.shape[1]
             }

smBlr = pystan.StanModel(model_code=blrModel)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_3e737708637464be719ee19b1a2c8195 NOW.


In [28]:
fitBlrNuts = smBlr.sampling(data=credit_dat, iter=1000, chains=4, algorithm='NUTS', control=dict(max_treedepth=15))

In [29]:
fitBlrNuts

Inference for Stan model: anon_model_3e737708637464be719ee19b1a2c8195.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

           mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
alpha     -1.22  1.8e-3   0.09   -1.4  -1.28  -1.22  -1.16  -1.05   2734    1.0
beta[1]   -0.75  1.7e-3   0.09  -0.94  -0.81  -0.75  -0.69  -0.59   2805    1.0
beta[2]    0.44  2.0e-3    0.1   0.23   0.37   0.44   0.51   0.65   2806    1.0
beta[3]   -0.41  2.0e-3   0.11  -0.62  -0.48  -0.41  -0.34   -0.2   2802    1.0
beta[4]    0.16  2.1e-3   0.11  -0.06   0.09   0.16   0.23   0.37   2653    1.0
beta[5]   -0.38  1.8e-3    0.1  -0.58  -0.45  -0.38  -0.32   -0.2   2901    1.0
beta[6]   -0.19  1.7e-3   0.09  -0.37  -0.25  -0.19  -0.13-7.8e-3   2919    1.0
beta[7]   -0.16  1.7e-3   0.08  -0.32  -0.21  -0.15   -0.1   0.01   2432    1.0
beta[8]    0.01  1.6e-3   0.09  -0.17  -0.05 8.7e-3   0.07    0.2   3440    1.0
beta[9]    0.

## Hierarchical bayesian logistic regression
Same beta and alpha coefficients as before but now also the variance parameter is given an exponential prior.

Furthermore interaction effects are added to the model which results to 300-dimensional vector of predictors and 300-dimensional vector of coefficients.

In [30]:
# Create new x-vector for the hierarchical model
from sklearn.preprocessing import PolynomialFeatures
hCreditX=PolynomialFeatures(2, interaction_only=True, include_bias=False).fit_transform(creditX)
print(hCreditX.shape, creditX.shape)
print(creditX[:2,])
print(hCreditX[:2,:30])

(1000, 300) (1000, 24)
[[-1.25456565 -1.23647786  1.38728734 -0.73343195  1.83316907  1.33807849
   0.44932648  1.04698668 -1.29372298  2.76645648  0.46083068  1.02707891
  -0.42828957  1.21459768 -0.19601428 -0.55270519 -0.33886163  0.32021217
  -0.20676767 -0.4669334   0.63444822 -0.14998296 -0.5         0.76635604]
 [-0.45902624  2.24819436 -0.74263181  0.96637654 -0.69970702 -0.31795924
  -0.96364986 -0.76597727 -1.29372298 -1.19140394  0.46083068 -0.704926
  -0.42828957 -0.82331789 -0.19601428 -0.55270519 -0.33886163  0.32021217
  -0.20676767 -0.4669334   0.63444822 -0.14998296 -0.5         0.76635604]]
[[-1.25456565 -1.23647786  1.38728734 -0.73343195  1.83316907  1.33807849
   0.44932648  1.04698668 -1.29372298  2.76645648  0.46083068  1.02707891
  -0.42828957  1.21459768 -0.19601428 -0.55270519 -0.33886163  0.32021217
  -0.20676767 -0.4669334   0.63444822 -0.14998296 -0.5         0.76635604
   1.55124265 -1.74044304  0.92013854 -2.29983094 -1.67870731 -0.56370956]
 [-0.45902624

In [31]:
# Bayesian logistic regression, hierarchical model
blrhModel = """
    data {
      int<lower=0> N;
      int<lower=0> K;   // number of predictors      
      matrix[N,K] x;
      int<lower=0,upper=1> y[N];
    }
    parameters {
      real alpha;
      vector[K] beta;
      real<lower=0> sigma2 ; 
    }
    model {
      sigma2 ~ exponential(rate = 0.01)
      alpha~normal(0,sigma2);
      beta~normal(0,sigma2);  
      y ~ bernoulli_logit(alpha + x*beta);
    }
"""

credit_dat = {'x': hCreditX,
                'y': creditY,
                'N': len(creditY),
                'K': hCreditX.shape[1]
             }

smBlrh = pystan.StanModel(model_code=blrModel)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_3e737708637464be719ee19b1a2c8195 NOW.


In [ ]:
fitBlrhNuts = smBlrh.sampling(data=credit_dat, iter=1000, chains=4, algorithm='NUTS', control=dict(max_treedepth=15))

In [ ]:
print(fitBlrhNuts)

In [ ]:
# 250-dimensional MVN
mvn250Model = """
    data {
        int wishSize;
        matrix[wishSize,wishSize] A;
        vector[wishSize] mean0;
    }
    parameters {
        vector[wishSize] theta;
    }
    model{
        theta ~ multi_normal(mean0, A);
    }

"""
from scipy.stats import wishart
wishSize=250
precisionMatrixA=wishart.rvs(df=wishSize, scale=np.identity(wishSize), random_state=None)

wishart_data={'A': precisionMatrixA, 'mean0': np.zeros(wishSize), 'wishSize': wishSize}

sm = pystan.StanModel(model_code=mvn250Model)

In [ ]:
fitWishartNuts = sm.sampling(data=wishart_data, iter=1000, chains=4, algorithm='NUTS')

In [ ]:
print(fitWishartNuts)